In [1]:
import cleo_tools as cleo
import numpy as np
import h5py

/home/fred/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
infile = open('data31_100k_LARGE.dat')
collisions = cleo.get_collisions(infile)

#for i in range(0,1):
#    collision = collisions[i]
#    pions,kaons,muons,electrons,photons = collision
#    print len(pions)

In [ ]:
'''
with h5py.File('cleo_test_large_zip','a') as hf:
    
    for i in xrange(0,len(collisions)):
        
        ev = 'event%d' % (i)
        
        event = hf.create_group(ev)
        pi = hf.create_group('%s/pions' % (ev))
        ka = hf.create_group('%s/kaons' % (ev))
        mu = hf.create_group('%s/muons' % (ev))
        e = hf.create_group('%s/electrons' % (ev))
        ph = hf.create_group('%s/photons' % (ev))
        
        collision = collisions[i]

        pions,kaons,muons,electrons,photons = collision

        for j in xrange(0,len(pions)):

            npions = 'pion%s' % (j) 

            pion = pions[j]

            pi.create_dataset(npions,data = pion,compression="gzip", compression_opts=9)

        for k in xrange(0,len(kaons)):

            nkaons = 'kaon%s' % (k) 

            kaon = kaons[k]

            ka.create_dataset(nkaons,data = kaon,compression="gzip", compression_opts=9)

        for l in xrange(0,len(muons)):

            nmuons = 'muon%s' % (l)

            muon = muons[l]

            mu.create_dataset(nmuons,data = muon,compression="gzip", compression_opts=9)

        for m in xrange(0,len(electrons)):

            nelectrons = 'electron%s' % (m)

            electron = electrons[m]
                
            e.create_dataset(nelectrons,data = electron,compression="gzip", compression_opts=9)

        for n in xrange(0,len(photons)):

            nphotons = 'photon%s' % (n)

            photon = photons[n]
            
            ph.create_dataset(nphotons,data = photon,compression="gzip", compression_opts=9)

'''
# This is a poor file structure. Due to the complex nature of the filesystem and the MASSIVE number of datasets
# file size increases dramatically. 

In [ ]:
!rm cleo_test

In [3]:
print '%s' % (npions)

NameError: name 'npions' is not defined

In [ ]:
with h5py.File('cleo_test_large') as hf:
    print hf.item
   